In [1]:
import pandas as pd 
import numpy as np
import os
np.seterr(divide='ignore', invalid='ignore')
import matplotlib.pyplot as plt
import math
import sys
import scipy.stats
import keras
import seaborn as sns
import json
import datetime

from numpy import array, newaxis
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from scipy import stats
from PIL import Image

%matplotlib inline

from Scoring import model
from Concat import get_concat_h_resize

seed = 5

# To read Score for each data instances and Raw Data

In [ ]:
Error = 'BAR_Error_metrics'
Score = 'Raw_Score_Path'

##  Data Load 13 - Error Metrics 

In [ ]:
y = pd.read_csv(Error)
X = y.iloc[:,0:13]
Y = np.load(Score)
Y=Y.astype(float)

##  Feature Selection

In [ ]:
corr_matrix = X.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
to_drop= [column for column in upper.columns if any(upper[column] > 0.9)]
training_data = np.array(X.drop(X[to_drop], axis=1))
training_data= training_data.reshape(training_data.shape[0], training_data.shape[1], 1)
testing_label = Y

In [ ]:
training_data.shape

In [ ]:
numOfRows = training_data.shape[1]
numOfColumns = training_data.shape[2]
reshapedSegments = training_data.reshape(training_data.shape[0], numOfRows, numOfColumns)

X_train, X_test, y_train, y_test = train_test_split(training_data, y ,test_size=0.2, random_state=seed)

# Calling the scoring Model

In [ ]:
model = model()
print(model.summary())
keras.utils.plot_model(model, show_shapes=True)

In [ ]:
history = model.fit(X_train,y_train,epochs=150,batch_size=32,validation_data=(X_test, y_test))

# Calculating Regression Metrics

In [ ]:
ypred = model.predict(X_test)
MSE = mean_squared_error(y_test, ypred)
R2Score = r2_score(y_test, ypred)


print(f"Mean Squared Error : ", MSE)
print(f"R-Squared Score : ", R2Score)

# Saving the predicted score and test raw score dataset and Regression Metrics

In [ ]:
if not os.path.exists('Scoring Results'):
    os.makedirs('Scoring Results')

In [ ]:
current_date_and_time = datetime.datetime.now()
current_date_and_time_string = str(current_date_and_time)

In [ ]:
dict_vals = {'Mean Squared Error': MSE,
                 'R-Squared Score': R2Score,
                 }

json_dump = json.dumps(dict_vals)
f = open("Scoring Results/Metrics_deep_learning_scoring_"+current_date_and_time_string+".json", "w")
f.write(json_dump)
f.close()

print(f"Successful!! Files stored")

In [ ]:
Person = ['Player1']

In [ ]:
ypred.to_csv("Scoring Results/Predicted_Score_"+Person"_"+current_date_and_time_string+".csv",index=False,header=None)

In [ ]:
y_test.to_csv("Scoring Results/Raw_Score_"+Person"_"+current_date_and_time_string+".csv",index=False,header=None)

# Correlation Plot between Predicted and Raw Scores

In [ ]:
result= np.concatenate((ypred, y_test), axis=1)
result = pd.DataFrame(result, columns = ['PREDICT SCORE', 'RAW SCORE'])

In [ ]:
sns.heatmap(result.corr(), annot = True ,vmin=0, vmax=1, cmap='ocean', center= 0, linewidths=3, linecolor='black', fmt='.3g', cbar=False)
ax = plt.plot()
sns.set(font_scale=2)
plt.xticks()
plt.yticks(rotation = 0)
ax.yaxis.set_label_coords(-0.1,1.02)
plt.savefig("Scoring Results/Correlation_Matrix.png", format='png',dpi=300,bbox_inches='tight')
plt.show()

# Concatenate correlation plots

In [ ]:
im2 = Image.open('Scoring Results/Correlation_Matrix1.png')
im1 = Image.open('Scoring Results/Correlation_Matrix2.png')

im3 = get_concat_h_resize(im2, im1, resample=Image.BICUBIC, resize_big_image=True).save('Scoring Results/Correlation_Uncontrolled.png')